In [1]:
import astropy.coordinates as coord
import numpy as np
from gala.units import UnitSystem
from yellowcard.model import TimingArgumentModel 
from yellowcard.coordinates import fiducial_m31_c
import astropy.units as u
from scipy.optimize import minimize

In [2]:
model = TimingArgumentModel(distance = fiducial_m31_c.distance,
                            pm = u.Quantity([fiducial_m31_c.pm_ra_cosdec, fiducial_m31_c.pm_dec]),
                            radial_velocity = fiducial_m31_c.radial_velocity,
                            tperi = 13.7*u.Gyr,
                            distance_err = 10*u.kpc,
                            pm_err = u.Quantity([10*u.microarcsecond/u.yr, 10*u.microarcsecond/u.yr]),
                            radial_velocity_err = 10 * u.km / u.s,
                            tperi_err = .25 * u.Gyr,
                            )

In [3]:
e = 0.5
eta = 5*u.rad
galcen_m31 = fiducial_m31_c.transform_to(model.galcen_frame)
galcen_m31_pos = galcen_m31.data.without_differentials()
galcen_m31_vel = galcen_m31.velocity
galcen_m31_L   = galcen_m31_pos.cross(galcen_m31_vel)
galcen_m31_L   = galcen_m31_L / galcen_m31_L.norm()

In [4]:
galcen_m31_L.xyz

<Quantity [-0.83720219, -0.3124973 ,  0.44881838]>

In [5]:
init_par = {}

init_par['a'] = fiducial_m31_c.distance.value/(1-e*np.cos(eta))
init_par['ecoseta'] = e*np.cos(eta)
init_par['esineta'] = e*np.sin(eta)
init_par['M'] = (4e12*u.Msun).decompose(model.unit_system).value
init_par['Lhatlg'] = galcen_m31_L.xyz


In [6]:
model.ln_likelihood(init_par)

-1655.3552205691667

In [7]:
model(model.pack_pars(init_par))

-1655.3552205691667

In [8]:
model.pack_pars(init_par)

<Quantity [ 8.51813663e+02,  1.41831093e-01, -4.79462137e-01,
            4.00000000e+00, -8.37202186e-01, -3.12497297e-01,
            4.48818381e-01]>

In [16]:
result = minimize( lambda *args: -model(*args), model.pack_pars(init_par), method='BFGS')

/Users/katiechambe/Documents/Research/projects/yellowCard/yellowcard/keplerianPlane.py:61: RuntimeWarning: invalid value encountered in sqrt
  return 2 * np.arctan( np.sqrt(A) * B )

/Users/katiechambe/Documents/Research/projects/yellowCard/yellowcard/keplerianPlane.py:53: RuntimeWarning: invalid value encountered in double_scalars
  B = ( 1 - self.e**2 )**(1/2)

/Users/katiechambe/miniconda3/envs/BADS/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0

/Users/katiechambe/Documents/Research/projects/yellowCard/yellowcard/keplerianPlane.py:61: RuntimeWarning: invalid value encountered in sqrt
  return 2 * np.arctan( np.sqrt(A) * B )

/Users/katiechambe/Documents/Research/projects/yellowCard/yellowcard/keplerianPlane.py:53: RuntimeWarning: invalid value encountered in double_scalars
  B = ( 1 - self.e**2 )**(1/2)

/Users/katiechambe/Documents/Research/projects/yellowCard/yellowcard/keplerianPlane.py:61: Run

In [17]:
result

      fun: 1533.1019163661115
 hess_inv: array([[ 1.02769990e+00,  1.82828031e-04, -6.31705487e-05,
        -1.74671699e-01, -1.58507297e-02,  2.97304271e-03,
         3.10570231e-02],
       [ 1.82828031e-04,  1.33527258e-04, -4.28622076e-05,
         1.14361053e-03, -8.23792606e-04, -5.29048810e-04,
         2.11612916e-05],
       [-6.31705487e-05, -4.28622076e-05,  7.21989497e-05,
        -1.35869408e-03, -2.86296741e-04, -2.79131570e-05,
         2.81714134e-04],
       [-1.74671699e-01,  1.14361053e-03, -1.35869408e-03,
         1.65887668e+00,  2.81665403e-01,  2.24154573e-02,
        -3.38942247e-01],
       [-1.58507297e-02, -8.23792606e-04, -2.86296741e-04,
         2.81665403e-01,  1.46754864e-01,  3.87924434e-02,
        -1.01372889e-01],
       [ 2.97304271e-03, -5.29048810e-04, -2.79131570e-05,
         2.24154573e-02,  3.87924434e-02,  1.93741640e-02,
        -2.10586577e-02],
       [ 3.10570231e-02,  2.11612916e-05,  2.81714134e-04,
        -3.38942247e-01, -1.01372889

In [18]:
model.unpack_pars(result.x)

{'a': array(852.03900205),
 'ecoseta': array(0.14351379),
 'esineta': array(-0.05635582),
 'M': array(2.77875582),
 'Lhatlg': array([-0.97837221, -0.25671605,  0.69034609])}

In [19]:
init_par

{'a': <Quantity 851.81366257>,
 'ecoseta': <Quantity 0.14183109>,
 'esineta': <Quantity -0.47946214>,
 'M': 3.999999999999999,
 'Lhatlg': <Quantity [-0.83720219, -0.3124973 ,  0.44881838]>}